<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/13_tilemaker_india_pbf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tilemaker

1. [Creating tiles from pbf files using `tilemaker`](https://github.com/systemed/tilemaker)

# Setup Environment

1. Install Pmtiles
2. GDAL
3. tilemaker

In [ ]:
%%shell

# Download go-pmtiles for Linux x86_64 quietly
wget -q https://github.com/protomaps/go-pmtiles/releases/download/v1.28.3/go-pmtiles_1.28.3_Linux_x86_64.tar.gz

# Verify SHA256 checksum
echo "06cf492adc2c7fcd23c4f11a98a5292f4cbe04d3afc3a6b38a07bb47452daca2 go-pmtiles_1.28.3_Linux_x86_64.tar.gz" | sha256sum --check --quiet

# Extract quietly
tar -xzf go-pmtiles_1.28.3_Linux_x86_64.tar.gz -C /tmp/ > /dev/null 2>&1

# Install binary to /usr/local/bin (assuming binary is named 'pmtiles')
sudo mv /tmp/pmtiles /usr/local/bin/pmtiles > /dev/null 2>&1

# Clean up
rm go-pmtiles_1.28.3_Linux_x86_64.tar.gz

# Check installation
echo "pmtiles Installation complete."
pmtiles version

## Installing `tilemaker`

In [ ]:
%%shell
set -euo pipefail
export DEBIAN_FRONTEND=noninteractive

apt-get update -qq
apt-get install -y -qq \
  git build-essential \
  libboost-dev libboost-filesystem-dev libboost-program-options-dev \
  libsqlite3-dev rapidjson-dev zlib1g-dev libshp-dev \
  lua5.3 liblua5.3-dev \
  protobuf-compiler libprotobuf-dev

cd /content
if [ ! -d tilemaker ]; then
  git clone --depth 1 https://github.com/systemed/tilemaker.git
fi

cd tilemaker
make -j"$(nproc)"
sudo make install

## Download OSM data for India

In [ ]:
%%shell
cd /content
wget -O india.osm.pbf https://download.geofabrik.de/asia/india-latest.osm.pbf

In [ ]:
%%shell
sudo apt install osmium-tool

## Filter OSM Data for Places only

In [ ]:
%%shell
osmium tags-filter india.osm.pbf nw/place r/place -o india_places.osm.pbf

In [ ]:
# %%shell
# sed -i 's/name:latin/name:en/g' /content/tilemaker/resources/process-openmaptiles.lua

40 mins to run this cell, when max zoom level is at 18

In [ ]:
%%shell
## Zoom level upto 17
wget https://raw.githubusercontent.com/kavyajeetbora/foursquare_ai/master/docs/config-openmaptiles.json -O /content/tilemaker/resources/config-openmaptiles.json

In [ ]:
%%shell
tilemaker /content/india_places.osm.pbf --output /content/india-latest-en.mbtiles --config /content/tilemaker/resources/config-openmaptiles.json --process /content/tilemaker/resources/process-openmaptiles.lua

In [ ]:
# from google.colab import files
# files.download('/content/india-latest-en.mbtiles')

Convert mbtiles to pmtiles

In [ ]:
%%shell
pmtiles convert india-latest-en.mbtiles india-latest-en.pmtiles

  64% || (12094761/18851001, 44488 it/s) 4s:2m31s]

In [ ]:
from google.colab import files
files.download('/content/india-latest-en.pmtiles')